In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px


In [ ]:
AZUL1, AZUL2, AZUL3, AZUL4, AZUL5 = '#03045e', '#0077b6', "#00b4d8", '#90e0ef', '#CDDBF3'
CINZA1, CINZA2, CINZA3, CINZA4, CINZA5 = '#212529', '#495057', '#adb5bd', '#dee2e6', '#f8f9fa'
VERMELHO1, LARANJA1, AMARELO1, VERDE1, VERDE2 = '#e76f51', '#f4a261',	'#e9c46a', '#4c956c', '#2a9d8f'

In [ ]:
data = ('Equities_ETFTradeFile_20230612_1.csv')
df = pd.read_csv(data, sep=';')

display(df)

In [ ]:
df.info()

In [ ]:
#DataFrame - Variação de Cotação Inicial dos ativos 

df['RptDt'] = pd.to_datetime(df['RptDt'], format='%Y-%m-%d')
df['RptDt'] = df['RptDt'].sort_values(ascending=True)


df['FrstPric'] = df['FrstPric'].sort_values(ascending=True)
df_filtred = df[df['FrstPric'] >= 100.00]
#display(df_filtred)

df_year_TckrSymb = df_filtred[['RptDt','TckrSymb','FrstPric']]
df_year_TckrSymb['year'] = df_filtred.RptDt.dt.year
df_year_TckrSymb.drop(columns='RptDt', inplace=True)
df_year_TckrSymb.sort_values(by='FrstPric', ascending=True, inplace=True)


display(df_year_TckrSymb)

In [ ]:
def gerar_grafico_variacao_cotacao():
    
    #Area do grafico
    fig, axs = plt.subplots(figsize=(16,4))
    fig.subplots_adjust(right=0.7)
    sns.set_theme(style='white')

    #Paleta de cores
    paletta = sns.color_palette('Blues', len(df_year_TckrSymb))
    
    #cores = [AZUL4, AZUL3, AZUL2, AZUL1,VERDE1, VERDE2]

    #Gerando o grafico
    axs = sns.barplot(data=df_year_TckrSymb, x='TckrSymb', y='FrstPric', palette = paletta, edgecolor= CINZA4)

    #Personalizando o gráfico
    axs.set_title('Variação da cotação inicial dos ativos em 2023', fontsize=18, color=CINZA2, loc='left', fontweight='bold')
    axs.text(0.0, 1, 'Top ETF'+"'s" +' acima de 100 reais',color=CINZA2, transform = axs.transAxes, fontsize=14, ha='left', va='top')
    axs.set_xlabel('ETF')
    axs.set_ylabel('')
    axs.set_yticklabels([])
    sns.despine(left = True, bottom = True)

    # Valores das barras
    for i, bar in enumerate(axs.patches):
        valor = bar.get_height()
        qtd = f'R$ {valor:,.1f}'.replace(',', '.')  
        axs.text(bar.get_x() + bar.get_width()/2, bar.get_height(), qtd, 
                color=CINZA2, fontsize=8, fontweight='bold', ha='center', va='bottom' )

    plt.show()
    
    return axs

gerar_grafico_variacao_cotacao()

In [184]:
#DataFrame - Volatilidade do mercado em 2023

df_filtred['Return'] = df_filtred.groupby('TckrSymb')['LastPric'].transform('last') - df_filtred.groupby('TckrSymb')['FrstPric'].transform('first')
display(df_filtred)

df_variation = df_filtred[['Return', 'TckrSymb', 'LastPric', 'FrstPric']]
display(df_variation)


,RptDt,TckrSymb,SctyId,SctySrc,MktIdrCd,FrstPric,MinPric,MaxPric,LastPric,OscnPctg,IndxVal,TradAvrgPric,PrvsDayClsgPric,DataSts,Return
4,2023-06-12,BCIC,200001070068,8,BVMF,115.73,115.51,116.45,116.18,0.0039,116.18,116.15,115.72,I,0.45
5,2023-06-12,BDEF,200001070193,8,BVMF,113.62,113.32,114.91,114.56,0.0084,114.56,114.30,113.60,I,0.94
6,2023-06-12,BDOM,200000952526,8,BVMF,102.19,102.19,103.31,103.06,0.0085,103.06,102.86,102.19,I,0.87
7,2023-06-12,BMMT,200001070019,8,BVMF,101.19,100.82,101.63,101.50,0.0031,101.50,101.31,101.18,I,0.31
8,2023-06-12,BOVA,200001030806,8,BVMF,113.28,112.98,113.95,113.58,0.0030,113.58,113.52,113.24,I,0.30
9,2023-06-12,BOVB,200000295865,8,BVMF,118.15,117.83,118.87,118.47,0.0027,118.47,118.41,118.15,I,0.32
11,2023-06-12,BOVV,200000892380,8,BVMF,118.62,118.30,119.33,118.93,0.0028,118.93,118.87,118.59,I,0.31
14,2023-06-12,BREW,200001070230,8,BVMF,114.79,114.64,115.65,115.41,0.0054,115.41,115.30,114.78,I,0.62
15,2023-06-12,BXPO,200000950776,8,BVMF,104.72,104.11,105.01,104.79,0.0008,104.79,104.64,104.70,I,0.07
18,2023-06-12,ECOO,10016355,8,BVMF,101.39,101.39,102.35,102.06,0.0066,102.06,101.98,101.39,I,0.67


,Return,TckrSymb,LastPric,FrstPric
4,0.45,BCIC,116.18,115.73
5,0.94,BDEF,114.56,113.62
6,0.87,BDOM,103.06,102.19
7,0.31,BMMT,101.50,101.19
8,0.30,BOVA,113.58,113.28
9,0.32,BOVB,118.47,118.15
11,0.31,BOVV,118.93,118.62
14,0.62,BREW,115.41,114.79
15,0.07,BXPO,104.79,104.72
18,0.67,ECOO,102.06,101.39


In [202]:
def gerar_grafico_variacao_preco():
    
    fig = px.line(
        data_frame= df_variation,
        x='TckrSymb',
        y='Return',
        markers=True,
        labels={'TckrSymb':'Ativo'},
        color_discrete_sequence=[AZUL2, VERMELHO1,AMARELO1,VERDE1, CINZA3]        
    )
    
    fig.update_yaxes(tickprefix="R$ ", ticksuffix=" centavos")
    
    fig.update_layout(
        width=1200, 
        height=600, 
        font_family = 'DejaVu Sans', 
        font_size=15, 
        font_color= CINZA2, 
        title_font_color= CINZA1, 
        title_font_size=24,
        title_text='Delta dos preços de cada ativo 2023',
        xaxis_title='', 
        yaxis_title='', 
        plot_bgcolor= CINZA5
    )
    
    fig.update_xaxes(tickvals=df_variation.TckrSymb)
    fig.update_traces(mode='markers+lines', hovertemplate ='<b>ATIVO:<b> %{x} <b>|<b> <b>VARIAÇÃO:<b> %{y}')
    fig.update_layout(hovermode='closest')    
    fig.update_layout(hovermode='x')          
    fig.update_layout(hovermode='x unified')  

    return fig

gerar_grafico_variacao_preco()